# EPTA GWB analyses tutorial - adapted from material by Stas Babak and Siyuan Chen

<a href="https://colab.research.google.com/github/golamshaifullah/EPTADR2_tutorial/blob/main/tutorials/03_Enterprise_GWB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Run the following two cells only when using colab! 

In [ ]:
# This cell will reset the kernel.
# Run this cell, wait until it's done, then run the next.
!pip install -q condacolab
import condacolab
condacolab.install_mambaforge()

In [ ]:
%%capture
!mamba install -y -c conda-forge enterprise_extensions la_forge corner "scipy<1.13"
!git clone https://github.com/golamshaifullah/EPTADR2_tutorial

### The actual notebook starts from here:

In [ ]:
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    homedir = '/content/EPTADR2_tutorial'
else:
    homedir = '../'

In [ ]:
from __future__ import division

import numpy as np
import os, glob, json
import matplotlib.pyplot as plt
import corner

import enterprise
from enterprise.pulsar import Pulsar
from enterprise.signals import utils
from enterprise_extensions import models, model_utils, hypermodel
from enterprise_extensions.sampler import JumpProposal

from PTMCMCSampler.PTMCMCSampler import PTSampler as ptmcmc
from types import SimpleNamespace

from concurrent.futures import ProcessPoolExecutor

In [ ]:
options = SimpleNamespace()
options.basedir =  f'{homedir}/data/EPTA_DR2'
options.dataset = 'DR2new+'
options.datadir = os.path.join(options.basedir, options.dataset)
options.noisedir = os.path.join(options.basedir, 'noisefiles_t2equad', options.dataset)
options.red_components =  0
options.dm_components =  0
options.chrom_components =  0
options.common_components =  30
options.common_psd =  'powerlaw'
options.common_components =  30
options.gamma_common =  None
options.red_components =  0
options.dm_components =  0
options.chrom_components =  0
options.num_dmdips =  2
options.bayesephem =  False
options.common_sin =  False
options.psrname =  'string'
options.resume =  False
options.emp =  None
options.number =  1e7
options.thin =  100
options.PsrList = ['J0613-0200','J1012+5307','J1600-3053','J1713+0747','J1744-1134','J1909-3744']
options.orf_bins = None
options.orf = 'crn'

# Load par+tim+noise files


In [ ]:
parfiles = sorted(glob.glob( options.datadir  + '/J*/*.par'))
timfiles = sorted(glob.glob( options.datadir  + '/J*/*_all.tim'))
noisefiles = sorted(glob.glob( options.noisedir  + '/*.json'))

parfiles = [x for x in parfiles if x.split('/')[-1].split('.')[0] in  options.PsrList ]
timfiles = [x for x in timfiles if x.split('/')[-1].split('_')[0] in  options.PsrList ]
noisefiles = [x for x in noisefiles if x.split('/')[-1].split('_')[0] in  options.PsrList ]

params = {}
for nf in noisefiles:
    with open(nf, 'r') as fin:
        params.update(json.load(fin))

def create_pulsar(parfile, timfile):
    return Pulsar(parfile, timfile, ephem='DE440')

# Create pulsar objects in parallel
psrs = []

if IN_COLAB:
    for par, tim in zip(parfile, timfiles):
        create_pulsar(par, tim)
else:
    with ProcessPoolExecutor() as executor:
        # Use a dictionary to associate parfiles with timfiles
        futures = {executor.submit(create_pulsar, p, t): (p, t) for p, t in zip(parfiles, timfiles)}
        for future in futures:
            psrs.append(future.result())

# Check the number of pulsars created
print(f"Number of pulsar objects created: {len(psrs)}")

In [ ]:
# load noise models and files
params = {}
for nf in noisefiles:
    with open(nf, 'r') as fin:
        params.update(json.load(fin))

if not options.red_components:
    try:
        red_dict = {}
        with open( options.noisedir  + '/red_dict.json','r') as rd:
            red_dict.update(json.load(rd))
    except:
        raise UserWarning('Custom pulsar red noise frequency components not set.')
else:
    red_dict = options.red_components

if not options.dm_components:
    try:
        dm_dict = {}
        with open( options.noisedir  + '/dm_dict.json','r') as dd:
            dm_dict.update(json.load(dd))
    except:
        raise UserWarning('Custom pulsar DM noise frequency components not set.')
else:
    dm_dict = options.dm_components

if not options.chrom_components:
    try:
        chrom_dict = {}
        with open( options.noisedir  + '/chrom_dict.json','r') as cd:
            chrom_dict.update(json.load(cd))
    except:
        raise UserWarning('Custom pulsar scattering noise frequency components not set.')
else:
    chrom_dict = options.chrom_components

try:
    gamma_common = float(options.gamma_common)
except:
    gamma_common = None

if options.psrname is not None:
    dropout = True
else:
    dropout = False

if options.orf_bins is not None:
    orf_bins = np.loadtxt(options.orf_bins)
else:
    orf_bins = None

Assuming purely GW emission driven circular binaries allows one to write the strain of the GWB to be
\begin{equation}
\large
h_c (f) = A_{GWB} f^{-2/3}
\end{equation}

The characteristic strain $h_c$ is connected to the induced correlated red noise between two pulsars $i$ and $j$ via the power spectral density (which is the Fourier transform of the common residuals $R_{ij}(t)$ between pulsars $i$ and $j$)
\begin{equation}
\large
S_{ij}(f) = \Gamma_{ij} \frac{h_c^2(f)}{12\pi^2 f^3}
\end{equation}
where $\Gamma_{ij}$ is the overlap reduction function and describes the degree of correlation between the noise in the pulsar pair $ij$, in case of an isotropic GWB it is the Hellings-Downs curve.

We can put $h_c$ into the PSD equation to get
\begin{equation}
\large
S_{ij}(f) = \Gamma_{ij} \frac{A_{GWB}^2 f^{-4/3}}{12\pi^2 f^3} = \frac{\Gamma_{ij}}{12\pi^2} A_{GWB}^2 f^{-13/3}
\end{equation}

White noise parameters are used fixed for the GWB analysis. See Gregory+Antoine tutorial on how to get EFAC+EQUAD.

To speed computation we usually assume that the overlap reduction function is just the identity matrix, ie. common red noise process with no spatial correlation. But HD correlated red noise search is done as a final confirmation.

When searching for a background, sometimes the $-\gamma=-13/3$ restriction is loosened to be $\gamma \in [0,7]$. This is equivalent to searching for a common red noise amongst all pulsars with a unknown spectral index and amplitude.

# GWB

In [ ]:
# create PTA object
pta = models.model_general(psrs, noisedict=params, orf=options.orf, 
                           gamma_common=13./3., upper_limit_common=True, 
                           bayesephem=True, dm_var=True)

In [ ]:
# draw initial sample
x0 = np.hstack(p.sample() for p in pta.params)

In [ ]:
x0

In [ ]:
# PTMCMC

# set output directory
outdir = '../chains/ptmcmc_test'

# save parameter names
np.savetxt(outdir+'/pars.txt', pta.param_names, fmt='%s')

ndim = len(x0)
N = int(1e4)

# initial jump covariance matrix
cov = np.diag(np.ones(ndim) * 0.01**2)

sampler = ptmcmc(ndim, pta.get_lnlikelihood, pta.get_lnprior, cov, outDir=outdir, resume=False)

# jump proposals
jp = JumpProposal(pta)
sampler.addProposalToCycle(jp.draw_from_prior,15)

In [ ]:
# SCAM = Single Component Adaptive Metropolis
# AM = Adaptive Metropolis
# DE = Differential Evolution
## You can keep all these set at default values
sampler.sample(x0, N, SCAMweight=30, AMweight=15, DEweight=50)

In [ ]:
chain = np.genfromtxt(f"{outdir}/chain_1.txt")
names = np.loadtxt(f"{outdir}/pars.txt",dtype=str)
chain = np.delete(chain,[chain.shape[1]-1,chain.shape[1]-2,chain.shape[1]-3,chain.shape[1]-4],1)
burn = int(0.25 * chain.shape[0])

In [ ]:
# plot chain and posterior

s = 7 # 7 ptmcmc, 1 single

fig = plt.figure(figsize=(15,5))
plt.subplot(121)
plt.title('{}'.format(names[-s]))
plt.plot(range(len(chain)),chain[:,-s])

plt.subplot(122)
uplim = 10.**np.percentile(chain[burn:,-s],95)
plt.hist(10.**chain[burn:,-s], 50, density=True, histtype='step', lw=2)
plt.axvline(uplim,label="{:.2e}".format(uplim))
plt.legend(loc=0)

In [ ]:
# corner plot
corner.corner(chain[burn:,[-7,-6]],labels=[names[-7],names[-6]],show_titles=1,quantiles=[0.05,0.5,0.95])
plt.show()

# GWB Model comparison

Here we use the `HyperModel` from `enterprise_extensions`. It creates a Bayesian hyper model with all parameters from the constituent models + a parameter `n_model` specifying which sub-model is being sampled. The fraction of steps that the sampler stays in model0 vs model1 give the odds-ratio between the two models.

This is typically used to gauge whether a common red noise process has a notable Bayes factor to be Hellings-Downs correlated.

In [ ]:
# create Hypermodel to compute Bayes factors between different models
pta = dict.fromkeys(np.arange(0, 2))

pta[0] = models.model_general(psrs, noisedict=params,
                              gamma_common=13./3., upper_limit_common=True,
                              bayesephem=False, dm_var=True)

pta[1] = models.model_general(psrs, noisedict=params, orf='hd', 
                              gamma_common=13./3., upper_limit_common=True,
                              bayesephem=False, dm_var=True)

super_model = hypermodel.HyperModel(pta)

outdir = f'{homedir}/chains/hyper_model_test/'
sampler = super_model.setup_sampler(resume=False, outdir=outdir)

In [ ]:
# sample hypermodel
N = int(1e4)
x0 = super_model.initial_sample()
sampler.sample(x0, N, SCAMweight=30, AMweight=15, DEweight=50)

In [ ]:
# Post processing
chain = np.genfromtxt(f'{homedir}/chains/hyper_model_test/chain_1.txt')
names = np.loadtxtf'{homedir}/chains/hyper_model_test/pars.txt',dtype=str)
burn = int(0.25 * chain.shape[0])

In [ ]:
# compute odds ratio between model 0 and 1
model_utils.odds_ratio(chain[burn:,-5])

In [ ]:
# corner plot
corner.corner(chain[burn:,[-6,-5]],labels=[names[-2],names[-1]],show_titles=1,quantiles=[0.05,0.5,0.95])
plt.show()